In [1]:
from SLIM_BPR.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
#from SLIM_ElasticNet.SLIMElasticNetRecommender import SLIMElasticNetRecommender

#from MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_BPR_Cython, MatrixFactorization_FunkSVD_Cython
#from MatrixFactorization.PureSVD import PureSVDRecommender

from Base.NonPersonalizedRecommender import TopPop, Random

from KNN.UserKNNCFRecommender import UserKNNCFRecommender
from KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender
#from GraphBased.RP3betaRecommender import RP3betaRecommender
#from GraphBased.P3alphaRecommender import P3alphaRecommender



In [2]:
import traceback, os
import scipy.sparse
URM_all = scipy.sparse.load_npz('URM_all_matrix.npz')
ICM_all = scipy.sparse.load_npz('ICM_all_matrix.npz')
URM_train = scipy.sparse.load_npz('URM_train_matrix.npz')
URM_test = scipy.sparse.load_npz('URM_test_matrix.npz')

In [3]:
recommender_list = [
    #Random,
    #TopPop,
    #P3alphaRecommender,
    #RP3betaRecommender,
    #ItemKNNCFRecommender,
    #UserKNNCFRecommender,
    #MatrixFactorization_BPR_Cython,
    #MatrixFactorization_FunkSVD_Cython,
    #PureSVDRecommender,
    SLIM_BPR_Cython,
    #SLIMElasticNetRecommender
    ]


from Base.Evaluation.Evaluator import SequentialEvaluator

evaluator = SequentialEvaluator(URM_test, [10], exclude_seen=True)


output_root_path = "result_experiments/"

# If directory does not exist, create
if not os.path.exists(output_root_path):
    os.makedirs(output_root_path)


logFile = open(output_root_path + "result_all_algorithms.txt", "a")


for recommender_class in recommender_list:

    try:

        print("Algorithm: {}".format(recommender_class))


        recommender = recommender_class(URM_train)
        recommender.fit(epochs=20,topK = 400, learning_rate=0.05 )
        results_run, results_run_string = evaluator.evaluateRecommender(recommender)
       
        print("Algorithm: {}, results: \n{}".format(recommender.__class__, results_run_string))
        logFile.write("Algorithm: {}, results: \n{}\n".format(recommender.__class__, results_run_string))
        logFile.flush()
        
    except Exception as e:
        traceback.print_exc()
        logFile.write("Algorithm: {} - Exception: {}\n".format(recommender_class, str(e)))
        logFile.flush()

Algorithm: <class 'SLIM_BPR.Cython.SLIM_BPR_Cython.SLIM_BPR_Cython'>
SLIM_BPR_Cython: Estimated memory required for similarity matrix of 20635 items is 1703.21 MB
Processed 1165777 ( 100.00% ) in 5.85 seconds. BPR loss is 2.37E+00. Sample per second: 199161
SLIM_BPR_Recommender: Epoch 1 of 20. Elapsed time 0.24 min
Processed 1165777 ( 100.00% ) in 5.76 seconds. BPR loss is 4.91E+00. Sample per second: 202325
SLIM_BPR_Recommender: Epoch 2 of 20. Elapsed time 0.32 min
Processed 1165777 ( 100.00% ) in 5.75 seconds. BPR loss is 6.57E+00. Sample per second: 202812
SLIM_BPR_Recommender: Epoch 3 of 20. Elapsed time 0.40 min
Processed 1165777 ( 100.00% ) in 5.70 seconds. BPR loss is 7.90E+00. Sample per second: 204590
SLIM_BPR_Recommender: Epoch 4 of 20. Elapsed time 0.48 min
Processed 1165777 ( 100.00% ) in 5.43 seconds. BPR loss is 8.98E+00. Sample per second: 214509
SLIM_BPR_Recommender: Epoch 5 of 20. Elapsed time 0.56 min
Processed 1165777 ( 100.00% ) in 5.66 seconds. BPR loss is 9.93E+00

'0.0848245'

In [25]:
import re
import pandas as pd



#Tuning Normalized
knn = [50, 100, 200, 300,400, 800]
learningRate = [0.1, 0.01, 0.025, 0.05, 0.075, 0.001]
iteration=0
resultDf = pd.DataFrame()


for toppK in knn:
    for lr in learningRate:
        print("X"*70)
        print("Iteration  "+str(iteration))
        iteration=iteration+1
        print("Recommender KNN is= {}, LearningRate is= {}".format(toppK, lr))
        recommender.fit(epochs=30,topK = toppK, learning_rate=lr )
        results_run, results_run_string = evaluator.evaluateRecommender(recommender)
        m = re.search('MAP: (.+?),', results_run_string)
        if m:
            found = m.group(1)
        resultDf.at[toppK, lr]=found
        print("-----MAP------")
        print(found)

#Creazione del CSV
resultDf

XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
Iteration  0
Recommender KNN is= 50, LearningRate is= 0.01
Processed 1165777 ( 100.00% ) in 5.11 seconds. BPR loss is 2.32E-01. Sample per second: 228220
SLIM_BPR_Recommender: Epoch 1 of 5. Elapsed time 0.22 min
Processed 1165777 ( 100.00% ) in 4.75 seconds. BPR loss is 5.91E-01. Sample per second: 245437
SLIM_BPR_Recommender: Epoch 2 of 5. Elapsed time 0.29 min
Processed 1165777 ( 100.00% ) in 6.03 seconds. BPR loss is 8.73E-01. Sample per second: 193324
SLIM_BPR_Recommender: Epoch 3 of 5. Elapsed time 0.38 min
Processed 1165777 ( 100.00% ) in 4.65 seconds. BPR loss is 1.12E+00. Sample per second: 250500
SLIM_BPR_Recommender: Epoch 4 of 5. Elapsed time 0.46 min
Processed 1165777 ( 100.00% ) in 5.85 seconds. BPR loss is 1.34E+00. Sample per second: 199281
SLIM_BPR_Recommender: Epoch 5 of 5. Elapsed time 0.55 min
SequentialEvaluator: Processed 9676 ( 100.00% ) in 7.95 seconds. Users per second: 1217
-----MAP------
0.

Processed 1165777 ( 100.00% ) in 5.86 seconds. BPR loss is 2.35E-01. Sample per second: 198808
SLIM_BPR_Recommender: Epoch 1 of 5. Elapsed time 0.21 min
Processed 1165777 ( 100.00% ) in 5.47 seconds. BPR loss is 5.93E-01. Sample per second: 213039
SLIM_BPR_Recommender: Epoch 2 of 5. Elapsed time 0.29 min
Processed 1165777 ( 100.00% ) in 4.52 seconds. BPR loss is 8.81E-01. Sample per second: 257882
SLIM_BPR_Recommender: Epoch 3 of 5. Elapsed time 0.36 min
Processed 1165777 ( 100.00% ) in 4.58 seconds. BPR loss is 1.12E+00. Sample per second: 254638
SLIM_BPR_Recommender: Epoch 4 of 5. Elapsed time 0.42 min
Processed 1165777 ( 100.00% ) in 4.64 seconds. BPR loss is 1.34E+00. Sample per second: 251426
SLIM_BPR_Recommender: Epoch 5 of 5. Elapsed time 0.49 min
SequentialEvaluator: Processed 9676 ( 100.00% ) in 7.34 seconds. Users per second: 1318
-----MAP------
0.0808765
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
Iteration  9
Recommender KNN is= 200, LearningRate 

Processed 1165777 ( 100.00% ) in 5.49 seconds. BPR loss is 5.92E-01. Sample per second: 212438
SLIM_BPR_Recommender: Epoch 2 of 5. Elapsed time 0.37 min
Processed 1165777 ( 100.00% ) in 5.56 seconds. BPR loss is 8.80E-01. Sample per second: 209669
SLIM_BPR_Recommender: Epoch 3 of 5. Elapsed time 0.46 min
Processed 1165777 ( 100.00% ) in 5.52 seconds. BPR loss is 1.12E+00. Sample per second: 211149
SLIM_BPR_Recommender: Epoch 4 of 5. Elapsed time 0.54 min
Processed 1165777 ( 100.00% ) in 5.70 seconds. BPR loss is 1.34E+00. Sample per second: 204346
SLIM_BPR_Recommender: Epoch 5 of 5. Elapsed time 0.62 min
SequentialEvaluator: Processed 9676 ( 100.00% ) in 9.53 seconds. Users per second: 1016
-----MAP------
0.0799681
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
Iteration  17
Recommender KNN is= 800, LearningRate is= 0.1
Processed 1165777 ( 100.00% ) in 10.91 seconds. BPR loss is 5.42E+00. Sample per second: 106846
SLIM_BPR_Recommender: Epoch 1 of 5. Elapsed time

In [3]:
from createSubmission import create_submission

recommender=SLIM_BPR_Cython(URM_all)
recommender.fit(epochs=20,topK = 200, learning_rate=0.01)

create_submission(recommender)

SLIM_BPR_Cython: Estimated memory required for similarity matrix of 20635 items is 1703.21 MB
Processed 1211791 ( 100.00% ) in 5.78 seconds. BPR loss is 2.68E-01. Sample per second: 209603
SLIM_BPR_Recommender: Epoch 1 of 20. Elapsed time 0.22 min
Processed 1211791 ( 100.00% ) in 5.68 seconds. BPR loss is 6.77E-01. Sample per second: 213493
SLIM_BPR_Recommender: Epoch 2 of 20. Elapsed time 0.30 min
Processed 1211791 ( 100.00% ) in 5.50 seconds. BPR loss is 9.93E-01. Sample per second: 220433
SLIM_BPR_Recommender: Epoch 3 of 20. Elapsed time 0.38 min
Processed 1211791 ( 100.00% ) in 5.33 seconds. BPR loss is 1.27E+00. Sample per second: 227148
SLIM_BPR_Recommender: Epoch 4 of 20. Elapsed time 0.46 min
Processed 1211791 ( 100.00% ) in 5.87 seconds. BPR loss is 1.51E+00. Sample per second: 206437
SLIM_BPR_Recommender: Epoch 5 of 20. Elapsed time 0.55 min
Processed 1211791 ( 100.00% ) in 6.31 seconds. BPR loss is 1.73E+00. Sample per second: 191974
SLIM_BPR_Recommender: Epoch 6 of 20. Elap

import re

results_run_string
m = re.search('MAP: (.+?),', results_run_string)
if m:
    found = m.group(1)
found